<a href="https://colab.research.google.com/github/timwu64/Generative-AI---LoRA-FineTuning/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRa
* Model: bert-base-cased
* Evaluation approach: accuracy
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
! pip install -q "datasets==2.15.0"
! pip install datasets
! pip install transformers
! pip install peft
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# Includes the relevant imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
import numpy as np
from torch.utils.data import DataLoader

SEED = 123

In [3]:
# Define the Foundation Model

base_model = "bert-base-cased"
peft_model_name = base_model+'-peft'

# Tokenize the dataset with the foundation model
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load a pretrained Hugging Face model that can be used for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Includes the relevant imports and loads a Hugging Face dataset that can be used for sequence classification.
# Load and preprocess a dataset
from datasets import load_dataset

# Define the dataset
dataset_name = "imdb"

# Set dataset range
dataset_range = 500

# Import the datasets and transformers packages
dataset = load_dataset(dataset_name)

dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
# utility
from IPython.display import display, Markdown

display(Markdown("#### Sample data set"))
display(dataset["test"].to_pandas().head(5))

#### Sample data set

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


In [6]:
# preprocess the dataset by returning tokenized examples
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Includes relevant imports and loads a Hugging Face tokenizer that can be used to prepare the dataset.
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# A subset of the full dataset to be used to reduce computational resources needed.
train_dataset = tokenized_datasets["train"].shuffle(seed=SEED).select(range(dataset_range))
eval_dataset = tokenized_datasets["test"].shuffle(seed=SEED).select(range(dataset_range))

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
def compute_metrics(eval_pred):
    """Returns the accuracy of a model by providing predictions and ground truth labels."""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions == labels).mean()}

In [8]:
from transformers import TrainingArguments, Trainer
# hyperparameters
lr = 1e-3
batch_size = 16
num_epochs = 6

# Define training args for base
training_args_base = TrainingArguments(
    output_dir='./foundation_results',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [9]:
trainer_base = Trainer(
    model=model,
    args=training_args_base,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [10]:
# Evaluate the pretrained model
# Accuracy classification metric is calculated using the dataset and pretrained model
base_results = trainer_base.evaluate()
print("Pre-trained Model:", base_results['eval_accuracy'])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Pre-trained Model: 0.486


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
from peft import LoraConfig, TaskType

# Initializes a Hugging Face PEST config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [12]:
# Includes the relveant imports
from peft import get_peft_model

# Creates a PEFT model using the config
peft_model = get_peft_model(model, lora_config)

In [13]:
# Define training args for peft
training_args_peft = TrainingArguments(
    output_dir='./prft_results',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


In [14]:
print('PEFT Model')
peft_model.print_trainable_parameters()

PEFT Model
trainable params: 38,402 || all params: 108,350,212 || trainable%: 0.03544247795288116


In [15]:
trainer_peft = Trainer(
    model=peft_model,
    args=training_args_peft,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [16]:
# Train the PRFT model

print('PEFT Model')
peft_model.print_trainable_parameters()
trainer_peft.train()

PEFT Model
trainable params: 38,402 || all params: 108,350,212 || trainable%: 0.03544247795288116


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.668553,0.596000
2,No log,0.584868,0.696000
3,No log,0.555748,0.722000
4,No log,0.400873,0.824000
5,No log,0.397361,0.830000
6,No log,0.374817,0.840000


TrainOutput(global_step=192, training_loss=0.5040878057479858, metrics={'train_runtime': 344.2882, 'train_samples_per_second': 8.714, 'train_steps_per_second': 0.558, 'total_flos': 789687078912000.0, 'train_loss': 0.5040878057479858, 'epoch': 6.0})

In [17]:
peft_results = trainer_peft.evaluate()
print("Compare the result for Pre-trained and PEFT Model:")
print("Pre-trained Model:", base_results['eval_accuracy'])
print("PEFT Model:", peft_results['eval_accuracy'])

Compare the result for Pre-trained and PEFT Model:
Pre-trained Model: 0.486
PEFT Model: 0.84


In [18]:
# Save the PEFT model, Fine-tuened parameters are saved to a separate directory. The saved weights directory should be the same home directory as the notebook file
modified_tokenizer_name = peft_model_name+"-tokenizer"
# Save our LoRA model & tokenizer results
peft_model.save_pretrained(peft_model_name)
tokenizer.save_pretrained(modified_tokenizer_name)

('bert-base-cased-peft-tokenizer/tokenizer_config.json',
 'bert-base-cased-peft-tokenizer/special_tokens_map.json',
 'bert-base-cased-peft-tokenizer/vocab.txt',
 'bert-base-cased-peft-tokenizer/added_tokens.json',
 'bert-base-cased-peft-tokenizer/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
import numpy as np
from torch.utils.data import DataLoader
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = "bert-base-cased"
peft_model_name = base_model+'-peft'
modified_tokenizer_name = peft_model_name+"-tokenizer"

SEED = 123

In [20]:
# load in the PEFT (LoRA) model
from peft import AutoPeftModelForSequenceClassification
inference_model = AutoPeftModelForSequenceClassification.from_pretrained(
    peft_model_name,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1}).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# LOAD the Saved PEFT model
tokenizer = AutoTokenizer.from_pretrained(modified_tokenizer_name)

In [22]:
# Includes the relevant imports and loads a Hugging Face dataset that can be used for sequence classification.
# Load and preprocess a dataset
from datasets import load_dataset

# Define the dataset
dataset_name = "imdb"

# Set dataset range
dataset_range = 1000

# Import the datasets and transformers packages
dataset = load_dataset(dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [23]:
test_range = 5

print("Test dataset size:", len(dataset["test"]))

# Correct way to shuffle and select a subset from the test split
test_dataset = dataset["test"].shuffle(seed=SEED).select(range(test_range))

# Now you can work with `test_dataset` as your subset
print("Subset Test dataset size:", len(test_dataset))

Test dataset size: 25000
Subset Test dataset size: 5


In [24]:
def predict(tokenizer, model, text):
    """Returns the logits, the predicted class id and the corresponding label."""
    inputs = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class = logits.argmax().item()

    return {'logits': logits, 'class_id': predicted_class, 'label': model.config.id2label[predicted_class]}

In [25]:
def compute_metrics(eval_pred):
    """Returns the accuracy of a model by providing predictions and ground truth labels."""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions == labels).mean()}

In [26]:
# sentiment function
def get_sentiment(text):
    inputs = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        logits = inference_model(**inputs).logits
        predicted_class = logits.argmax().item()
    return predicted_class

### View the results

Let's look at few examples with labels and predicted values.

In [27]:
# Evaluating the PEFT (LoRA) model with sentiment prediction

# Get predictions for each test sample
pred_logits = []
for sample in test_dataset:
    logits = predict(tokenizer, inference_model, sample['text'])['logits']
    print("Input:", sample['text'].replace("<br />", " "), "Label:", sample['label'], "Prediction:", get_sentiment(sample['text']), "\n")


Input: Whatever you do, don't stop watching when you think the movie is over! Hang around for the first batch of credits or you'll really miss something! We saw this movie at the Savanaah film festival and thought it was the best of the bunch. Dreya Weber is a marvel really, not only because of her performance, but because she can pull it off so far above the ground. At the Q&A she said there were no wires or effects, so everything you see is really her going for it. Addie couldn't make it to the festival because she was dancing with Madonna. She was excellent and, my gawd, so beautiful. I was amazed that the film went over so well with the blue haired lady crowd, but there you have it, Savannnah isn't a backwater. Label: 1 Prediction: 1 

Input: Maybe we Aussies just have a totally different sense of humour and therein my lie the only problem here. I have a database of all the DVDs I own (including those received as gifts - which this was) and so, when entering a new one, I always ref

### Compare the accuracy between PFET (LoRA) and Pretrained model @ inference

Take 500 samples and compare the PFET (LoRA) and Pretrained model accuracy

In [28]:
test_range = 500

print("Test dataset size:", len(dataset["test"]))

# Correct way to shuffle and select a subset from the test split
test_dataset = dataset["test"].shuffle(seed=SEED).select(range(test_range))

# Now you can work with `test_dataset` as your subset
print("Subset Test dataset size:", len(test_dataset))

Test dataset size: 25000
Subset Test dataset size: 500


In [29]:
from tqdm import tqdm

# Assuming the functions `predict` and `compute_metrics` are defined elsewhere
# and 'test_dataset' is your dataset.

pred_logits = []
# Wrap test_dataset with tqdm to create a progress bar
for sample in tqdm(test_dataset, desc="Evaluating PEFT Model"):
    logits = predict(tokenizer, inference_model, sample['text'])['logits']
    pred_logits.append(logits.detach().cpu().squeeze().numpy())

# Compute accuracy
LoRA_model_accuracy = compute_metrics((pred_logits, np.array(test_dataset['label'])))
print("LoRA Model Accuracy:", LoRA_model_accuracy)

Evaluating PEFT Model: 100%|██████████| 500/500 [00:19<00:00, 25.38it/s]

LoRA Model Accuracy: {'accuracy': 0.84}


In [30]:
# Load the pretrain_model

# Tokenize the dataset with the foundation model
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load a pretrained Hugging Face model that can be used for sequence classification
pretrain_model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# Evaluating the pretrain_model

# Get predictions for each test sample
pred_logits = []
# Wrap test_dataset with tqdm to create a progress bar
for sample in tqdm(test_dataset, desc="Evaluating Pretrain Model"):
    logits = predict(tokenizer, pretrain_model, sample['text'])['logits']
    pred_logits.append(logits.detach().cpu().squeeze().numpy())

# Compute accuracy
pretrain_model_accuracy = compute_metrics((pred_logits, np.array(test_dataset['label'])))
print("Pretrain Model Accuracy:", pretrain_model_accuracy)

Evaluating Pretrain Model: 100%|██████████| 500/500 [00:19<00:00, 25.86it/s]


Pretrain Model Accuracy: {'accuracy': 0.488}


In [32]:
# Summary

print("Summary:")
print(f"The accuracy of the model has significantly enhanced following the LoRA, transitioning from an initial {pretrain_model_accuracy} with the pre-trained model to {LoRA_model_accuracy} in the PEFT (LoRA) enhanced model.")

Summary:
The accuracy of the model has significantly enhanced following the LoRA, transitioning from an initial {'accuracy': 0.488} with the pre-trained model to {'accuracy': 0.84} in the PEFT (LoRA) enhanced model.
